In [156]:
#FINAL FINAL FINAL

import cvxpy as cp
import numpy as np

# Data setup
num_students = 6
num_universities = 3

student_data = np.array([[3.9, 1500, 1],  # Student 1 - rank 3
                         [4.0, 1600, 1],  # Student 2 - rank 1
                         [4.0, 1600, 1],  # Student 3 - rank 2
                         [3.6, 1350, 5],  # Student 4 - rank 5 
                         [3.7, 1450, 1],  # Student 5 - rank 4 
                         [3.9, 1550, 1]])  # Student 6 - rank 6

university_requirements = np.array([[3.0, 1200, 20],  # University 1
                                    [3.5, 1300, 20],  # University 2
                                    [3.2, 1250, 20]]) # University 3


preference_scores = np.array([[3, 2, 1], 
                               [3, 2, 1],
                               [3, 2, 1],
                               [3, 2, 1],
                               [3, 2, 1],
                               [3, 2, 1]])

university_capacities = np.array([10, 10, 10])

x = cp.Variable((num_students, num_universities), boolean=True)

# Sort students
sorted_indices = np.lexsort((student_data[:, 2], student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []

#Eligibility constraint
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank:
            eligibility_matrix[i, j] = 1  
            constraints.append(x[i, j] == 1)  
        else:
            constraints.append(x[i, j] == 0)  

# Capacity Constraint
for j in range(num_universities):
    constraints.append(cp.sum(x[:, j]) <= university_capacities[j])

problem = cp.Problem(objective, constraints)
problem.solve()

print("Total preference score:", objective.value)
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    if admitted_universities:
        print(f"Student {original_index + 1} admitted to University/Universities: {', '.join(map(str, admitted_universities))}")
    else:
        print(f"Student {original_index + 1} not admitted to any university")

Total preference score: 36.0
Matching results:
Student 2 admitted to University/Universities: 1, 2, 3
Student 3 admitted to University/Universities: 1, 2, 3
Student 1 admitted to University/Universities: 1, 2, 3
Student 6 admitted to University/Universities: 1, 2, 3
Student 5 admitted to University/Universities: 1, 2, 3
Student 4 admitted to University/Universities: 1, 2, 3
